In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from app.database import SessionLocal, Base, engine
from app.models import *
from app import models
from app.utils import hash_password, reset_db


import random
import uuid
from typing import List, Dict, Set, Tuple

import numpy as np
from faker import Faker
from app.models import (
    User,
    Group,
    GroupMembership,
    Report,
    Contest,
    ContestParticipation,
    Role,
)

In [2]:
def random_subset(n, k):
    return random.sample([i for i in range(n)], k)

def pick_random_subset(population: list, k: int) -> list:
    """k distinct elements from population (k may be 0)."""
    if k == 0:
        return []
    return random.sample(population, k)

def get_mock_contest_standings(cid, include_users=None):
    if include_users is None:
        include_users = []
    res = {
        'contest': {
            'id': cid,
            'name': f'Codeforces Round {cid}',
            'type': 'CF',
            'phase': 'FINISHED',
            'frozen': False,
            'durationSeconds': 7200,
            'startTimeSeconds': 1746110100,
            'relativeTimeSeconds': 267545
        },
        'standings': [
        ]
    }
    num_participants = 8000
    standings = [
        {'handle': 'random-user'+str(i+1), 'rank': (i+1), 'points': 9754 - i, 'penalty': 0}
        for i in range(num_participants)
    ]
    res['standings'] = standings
    udx = pick_random_subset([i for i in range(num_participants)], len(include_users))

    for i in range(len(udx)):
        res['standings'][udx[i]]['handle'] = include_users[i]
    
    return res


In [3]:
"""
    simulate what happens at the actual database level when:
    
        - actual users register
        - a group is created by a user
        - a group moderator invites someone to join a group
        - a user requests to join a group
        - a group moderator accepts a pending join request
        - a user accepts a group join invite
        - a user is kicked out of a group
        - a user leaves a group

        - cron job pulls an upcoming contest from codeforces (or any other platform) - contests will be available for registration AFTER it gets pulled and added to db
        - a user registers for a contest within a group
        - a user revokes his contest registration within a group (can only be performed if time.now() < contest.start_date)
        - cron job pulls contest standings from cf and initiates rating recalc per group


        - a user creates a report object
        - a moderator/admin resolves a report object
        - ratings rollback for successful reports?
        
        - an announcement is created by siteadmins
"""

'\n    simulate what happens at the actual database level when:\n\n        - actual users register\n        - a group is created by a user\n        - a group moderator invites someone to join a group\n        - a user requests to join a group\n        - a group moderator accepts a pending join request\n        - a user accepts a group join invite\n        - a user is kicked out of a group\n        - a user leaves a group\n\n        - cron job pulls an upcoming contest from codeforces (or any other platform) - contests will be available for registration AFTER it gets pulled and added to db\n        - a user registers for a contest within a group\n        - a user revokes his contest registration within a group (can only be performed if time.now() < contest.start_date)\n        - cron job pulls contest standings from cf and initiates rating recalc per group\n\n\n        - a user creates a report object\n        - a moderator/admin resolves a report object\n        - ratings rollback for 

In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
faker = Faker()

NUM_USERS = 5_000
NUM_GROUPS = 30
NUM_CONTESTS = 5
NUM_PARTICIPATIONS = 15_000
NUM_REPORTS = 100
NUM_ANNOUNCEMENTS = 30

DEFAULT_PASS = "devpass"

In [5]:
reset_db()
Base.metadata.create_all(bind=engine)

db = SessionLocal()

dropping all tables...
all tables dropped.
creating tables from models...
schema rebuilt.


In [6]:
# POPULATE USERS
# create admins and common group
admin_users = [
    User(
        user_id="shrey",
        role=Role.admin,
        cf_handle="negative-xp",
        email_id="dwxghcqcd@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
    User(
        user_id="ani",
        role=Role.admin,
        cf_handle="roomTemperatureIQ",
        email_id="chjd7639383@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
]

users = admin_users

# create 5000 users
while len(users) < NUM_USERS:
    uid = 'testUser' + str(len(users)-1)
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=uid + '_cf',
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            email_id=str(uid) + '@gmail.com',
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )

db.add_all(users)
db.commit()

user_df = pd.read_sql("SELECT * FROM users", engine)
print(user_df.shape)
user_df.head()

(5000, 11)


,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,xNH8ZgKD8mMKTNoC8Qwj4ZLELXYBVtedDa+dIDcA8+UPUJ...,2025-05-18 15:23:53.338592
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,IMDHToMxUb43ncPXNf4DeWA6meFznZsLDFg9oX+MPomhsc...,2025-05-18 15:23:53.338592
2,testUser1,user,testUser1_cf,testUser1_ac,None,None,True,28,testUser1@gmail.com,EZRAMFJ7IERs8dLVHC1OvpBfDhNWTJFP2uDwme9uZsej3o...,2025-05-18 15:23:53.338592
3,testUser2,user,testUser2_cf,None,None,testUser2_tw,True,69,testUser2@gmail.com,EAA2TudueoURdI+aAgXBGQzG0MVhYs06e0DHBRYNMSLxUI...,2025-05-18 15:23:53.338592
4,testUser3,user,testUser3_cf,None,None,None,True,27,testUser3@gmail.com,ob/xrfD+RWvJZXRGl60eN+rbBowIhW3e07+SK+peLjzInA...,2025-05-18 15:23:53.338592


In [7]:
# POPULATE GROUPS

common_group = Group(
    group_id="main",
    group_name="main",
    group_description="group consisting of ALL users",
    is_private=False,
)

groups = [common_group]
for g_idx in range(1, NUM_GROUPS):
    g_id = f"g{g_idx:02d}"
    g_name = faker.unique.catch_phrase().lower().replace(" ", "‑")
    is_priv = random.random() < 0.3
    group = Group(
        group_id=g_id,
        group_name=g_name,
        group_description=faker.sentence(nb_words=10),
        is_private=is_priv,
    )
    groups.append(group)

db.add_all(groups)
db.commit()

group_df = pd.read_sql("SELECT * FROM groups", engine)
print(group_df.shape)
group_df.head()

(30, 5)


,group_id,group_name,group_description,is_private,timestamp
0,main,main,group consisting of ALL users,False,2025-05-18 15:23:53.555023
1,g01,sharable‑bifurcated‑algorithm,Each cause bill scientist nation opportunity a...,False,2025-05-18 15:23:53.555023
2,g02,robust‑4thgeneration‑open‑architecture,Respond red information last everything thank ...,False,2025-05-18 15:23:53.555023
3,g03,optimized‑global‑focus‑group,Democratic shake bill here grow gas enough.,False,2025-05-18 15:23:53.555023
4,g04,balanced‑upward-trending‑knowledgebase,By two bad fall pick those gun court attorney ...,False,2025-05-18 15:23:53.555023


In [8]:
# POPULATE MEMBERSHIPS
memberships = []


# common group should have ALL users
memberships.append(GroupMembership(
    user_id=users[0].user_id,
    group_id=groups[0].group_id,
    role=Role.admin,
    user_group_rating=1500,
    user_group_max_rating=1500,
))

for i in range(1, NUM_USERS):
    memberships.append(
        GroupMembership(
            user_id=users[i].user_id,
            group_id=groups[0].group_id,
            role=Role.user,
            user_group_rating=1500,
            user_group_max_rating=1500,
        )
    )

# add memberships to other groups
size_palette = [5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597]
while len(size_palette) < NUM_GROUPS - 1:
    size_palette.append(random.randint(10, 800))
random.shuffle(size_palette)

user_idx = [i for i in range(NUM_USERS)]

for i in range(1, NUM_GROUPS):
    grp_size = size_palette[i-1]
    members = pick_random_subset(user_idx, grp_size)
    # admin for this group
    memberships.append(
        GroupMembership(
            user_id = users[members[0]].user_id,
            group_id = groups[i].group_id,
            role=Role.admin,
            user_group_rating=1500,
            user_group_max_rating=1500, 
        )
    )
    for j in members[1:]:
        memberships.append(
            GroupMembership(
                user_id=users[j].user_id,
                group_id=groups[i].group_id,
                role=Role.user,
                user_group_rating=1500,
                user_group_max_rating=1500,
            )
        )

db.add_all(memberships)
db.commit()

membership_df = pd.read_sql("SELECT * FROM group_memberships", engine)
print(membership_df.shape)
membership_df.head()

(15335, 7)


,user_id,group_id,role,user_group_rating,user_group_max_rating,status,timestamp
0,shrey,main,admin,1500,1500,active,2025-05-18 15:23:53.579085
1,ani,main,user,1500,1500,active,2025-05-18 15:23:53.579085
2,testUser1,main,user,1500,1500,active,2025-05-18 15:23:53.579085
3,testUser2,main,user,1500,1500,active,2025-05-18 15:23:53.579085
4,testUser3,main,user,1500,1500,active,2025-05-18 15:23:53.579085


In [9]:
# populate CONTESTS]

NUM_CONTESTS = 30
contests = []
for i in range(NUM_CONTESTS):
    contests.append(
        Contest(
            contest_id = f"c{3000+i}",
            contest_name = f'Codeforces Contest {int(random.random() * 99999)}',
            platform = "Codeforces",
            start_time_posix = int(random.random() * 1e8),
            duration_seconds = 1938473,
            link = 'OCD',
            finished=bool(int(random.random() * 2)),
            internal_contest_identifier = f'{random.random() * 99999}',
        )
    )

db.add_all(contests)
db.commit()

contest_df = pd.read_sql("SELECT * FROM contests", engine)
print(contest_df.shape)
contest_df.head()

(30, 10)


,contest_id,contest_name,platform,start_time_posix,duration_seconds,link,internal_contest_identifier,standings,finished,timestamp
0,c3000,Codeforces Contest 13510,Codeforces,37750653,1938473,OCD,1318.9461077942383,None,False,2025-05-18 15:23:56.210459
1,c3001,Codeforces Contest 79832,Codeforces,77193911,1938473,OCD,21975.680304251066,None,False,2025-05-18 15:23:56.210459
2,c3002,Codeforces Contest 67124,Codeforces,2749703,1938473,OCD,29104.67876725598,None,True,2025-05-18 15:23:56.210459
3,c3003,Codeforces Contest 31286,Codeforces,61508801,1938473,OCD,22602.314963747525,None,True,2025-05-18 15:23:56.210459
4,c3004,Codeforces Contest 1775,Codeforces,69164192,1938473,OCD,10635.553068130106,None,True,2025-05-18 15:23:56.210459


In [10]:
# populate contest participations
participations = []

for contest in contests:
    for group in groups:
        members = group.memberships
        n_members = len(members)
        parts = random_subset(n_members, int(3*n_members//4))
        for ii in parts:
            participations.append(
                ContestParticipation(
                    user_id = members[ii].user_id,
                    group_id = group.group_id,
                    contest_id = contest.contest_id,
                    rating_before = members[ii].user_group_rating,
                )
            )

db.add_all(participations)
db.commit()

participation_df = pd.read_sql("SELECT * FROM contest_participations", engine)
print(participation_df.shape)
participation_df.head()

(344640, 7)


,user_id,group_id,contest_id,rank,rating_before,rating_after,timestamp
0,testUser4751,main,c3000,None,1500,None,2025-05-18 15:23:56.254511
1,testUser1024,main,c3000,None,1500,None,2025-05-18 15:23:56.254511
2,testUser2582,main,c3000,None,1500,None,2025-05-18 15:23:56.254511
3,testUser3098,main,c3000,None,1500,None,2025-05-18 15:23:56.254511
4,testUser4908,main,c3000,None,1500,None,2025-05-18 15:23:56.254511


In [11]:
# populate reports
n_parts = len(participations)
reports = []

while len(reports) < NUM_REPORTS:
    idx = int(random.random() * n_parts)
    rp = participations[idx]
    
    grp = db.query(Group).filter(Group.group_id == rp.group_id).all()[0]
    members = grp.memberships
    reporter = members[int(len(members) * random.random())]
    reports.append(
        Report(
            report_id = f"report{len(reports)}",
            group_id = rp.group_id,
            contest_id = rp.contest_id,
            reporter_user_id = reporter.user_id,
            respondent_user_id = rp.user_id,
            report_description = faker.sentence(nb_words=12)
        )
    )

db.add_all(reports)
db.commit()

report_df = pd.read_sql("SELECT * FROM reports", engine)
print(report_df.shape)
report_df.head()

(100, 10)


,report_id,group_id,contest_id,reporter_user_id,respondent_user_id,report_description,resolved,resolved_by,resolve_message,timestamp
0,report0,main,c3003,testUser3322,testUser4320,Voice boy wife condition while enter board its...,False,None,None,2025-05-18 15:24:21.996462
1,report1,main,c3022,testUser2998,testUser4289,Tonight couple and job mind southern rather vo...,False,None,None,2025-05-18 15:24:21.996462
2,report2,main,c3000,testUser2892,testUser49,Finish summer rest feel finally impact I fast ...,False,None,None,2025-05-18 15:24:21.996462
3,report3,g26,c3002,testUser3721,testUser2208,Fight decision size parent focus kid put.,False,None,None,2025-05-18 15:24:21.996462
4,report4,g08,c3008,testUser2082,testUser4462,List top somebody college be middle plan frien...,False,None,None,2025-05-18 15:24:21.996462


In [12]:
# populate announcememts

NUM_ANNOUNCEMENTS = 40
announcements = []
for i in range(NUM_ANNOUNCEMENTS):
    announcements.append(
        Announcement(
            announcement_id = f"anmt{i}",
            group_id = "main",
            title = faker.sentence(nb_words=7),
            content = faker.sentence(nb_words=100)
        )
    )

db.add_all(announcements)
db.commit()

announcement_df = pd.read_sql("SELECT * FROM announcements", engine)
print(announcement_df.shape)
announcement_df.head()

(40, 5)


,announcement_id,group_id,title,content,timestamp
0,anmt0,main,Consider whom item treat area buy check clearl...,Generation wait thus suffer economy play nearl...,2025-05-18 15:24:22.843017
1,anmt1,main,No guy eye hit late near stay perhaps particul...,Window hour some fund voice sense current meet...,2025-05-18 15:24:22.843017
2,anmt2,main,Machine whatever everything fear walk word sid...,First give value somebody event business quali...,2025-05-18 15:24:22.843017
3,anmt3,main,Reduce tree serious soon stay seven quite.,Their bank land region back nor article natura...,2025-05-18 15:24:22.843017
4,anmt4,main,Help painting always authority source onto.,Describe decade trade field training deep coup...,2025-05-18 15:24:22.843017


In [13]:
db.rollback()

In [14]:
pd.read_sql("SELECT * from contests", engine).head()

,contest_id,contest_name,platform,start_time_posix,duration_seconds,link,internal_contest_identifier,standings,finished,timestamp
0,c3000,Codeforces Contest 13510,Codeforces,37750653,1938473,OCD,1318.9461077942383,None,False,2025-05-18 15:23:56.210459
1,c3001,Codeforces Contest 79832,Codeforces,77193911,1938473,OCD,21975.680304251066,None,False,2025-05-18 15:23:56.210459
2,c3002,Codeforces Contest 67124,Codeforces,2749703,1938473,OCD,29104.67876725598,None,True,2025-05-18 15:23:56.210459
3,c3003,Codeforces Contest 31286,Codeforces,61508801,1938473,OCD,22602.314963747525,None,True,2025-05-18 15:23:56.210459
4,c3004,Codeforces Contest 1775,Codeforces,69164192,1938473,OCD,10635.553068130106,None,True,2025-05-18 15:23:56.210459


In [15]:
# update ratings for a contest across ALL groups

# TODO

In [16]:
pd.read_sql("SELECT * from users", engine).head()

,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,xNH8ZgKD8mMKTNoC8Qwj4ZLELXYBVtedDa+dIDcA8+UPUJ...,2025-05-18 15:23:53.338592
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,IMDHToMxUb43ncPXNf4DeWA6meFznZsLDFg9oX+MPomhsc...,2025-05-18 15:23:53.338592
2,testUser1,user,testUser1_cf,testUser1_ac,None,None,True,28,testUser1@gmail.com,EZRAMFJ7IERs8dLVHC1OvpBfDhNWTJFP2uDwme9uZsej3o...,2025-05-18 15:23:53.338592
3,testUser2,user,testUser2_cf,None,None,testUser2_tw,True,69,testUser2@gmail.com,EAA2TudueoURdI+aAgXBGQzG0MVhYs06e0DHBRYNMSLxUI...,2025-05-18 15:23:53.338592
4,testUser3,user,testUser3_cf,None,None,None,True,27,testUser3@gmail.com,ob/xrfD+RWvJZXRGl60eN+rbBowIhW3e07+SK+peLjzInA...,2025-05-18 15:23:53.338592


In [17]:
from sqlalchemy import func
from sqlalchemy.orm import Session
from typing import List, Tuple
from app import models

def get_group_member_counts(db: Session) -> List[Tuple[str, int]]:
    """
    Returns a list of (group_id, member_count) for all groups.
    Groups with 0 members are included.
    """
    results = (
        db.query(
            models.Group.group_id,
            func.count(models.GroupMembership.user_id).label("member_count")
        )
        .outerjoin(models.GroupMembership, models.Group.group_id == models.GroupMembership.group_id)
        .group_by(models.Group.group_id)
        .all()
    )
    return results


get_group_member_counts(db)


[('g10', 89),
 ('g01', 987),
 ('main', 5000),
 ('g20', 5),
 ('g05', 13),
 ('g06', 34),
 ('g23', 11),
 ('g29', 377),
 ('g04', 8),
 ('g09', 562),
 ('g24', 311),
 ('g02', 21),
 ('g28', 209),
 ('g14', 671),
 ('g15', 217),
 ('g03', 144),
 ('g13', 306),
 ('g11', 718),
 ('g22', 763),
 ('g19', 233),
 ('g18', 401),
 ('g16', 55),
 ('g07', 767),
 ('g08', 1597),
 ('g12', 28),
 ('g21', 610),
 ('g17', 210),
 ('g27', 319),
 ('g25', 206),
 ('g26', 463)]

In [18]:
db.rollback()

In [19]:
res = (
        db.query(
            models.Group,
            func.count(models.GroupMembership.user_id).label("member_count")
        )
        .outerjoin(models.GroupMembership, models.Group.group_id == models.GroupMembership.group_id)
        .group_by(models.Group.group_id)
        .all()
    )

In [20]:
res

[(<Group(id=g10, name='mandatory‑bifurcated‑frame')>, 89),
 (<Group(id=g01, name='sharable‑bifurcated‑algorithm')>, 987),
 (<Group(id=main, name='main')>, 5000),
 (<Group(id=g20, name='configurable‑contextually-based‑architecture')>, 5),
 (<Group(id=g05, name='focused‑bandwidth-monitored‑implementation')>, 13),
 (<Group(id=g06, name='public-key‑upward-trending‑encryption')>, 34),
 (<Group(id=g23, name='innovative‑stable‑benchmark')>, 11),
 (<Group(id=g29, name='focused‑directional‑algorithm')>, 377),
 (<Group(id=g04, name='balanced‑upward-trending‑knowledgebase')>, 8),
 (<Group(id=g09, name='right-sized‑asymmetric‑info-mediaries')>, 562),
 (<Group(id=g24, name='synergized‑scalable‑firmware')>, 311),
 (<Group(id=g02, name='robust‑4thgeneration‑open‑architecture')>, 21),
 (<Group(id=g28, name='user-centric‑dynamic‑flexibility')>, 209),
 (<Group(id=g14, name='face-to-face‑asymmetric‑graphical‑user‑interface')>, 671),
 (<Group(id=g15, name='automated‑full-range‑archive')>, 217),
 (<Group(i

In [21]:
res[0][0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x110de0c50>,
 'group_description': 'Citizen about reveal rest will seven medical blood personal success medical.',
 'timestamp': datetime.datetime(2025, 5, 18, 15, 23, 53, 555023),
 'is_private': False,
 'group_name': 'mandatory‑bifurcated‑frame',
 'group_id': 'g10'}

In [22]:
res[0][1]

89

In [23]:
pd.read_sql("SELECT * from users", engine).head()

,user_id,role,cf_handle,atcoder_handle,codechef_handle,twitter_handle,internal_default_rated,trusted_score,email_id,hashed_password,timestamp
0,shrey,admin,negative-xp,None,None,None,True,88,dwxghcqcd@gmail.com,xNH8ZgKD8mMKTNoC8Qwj4ZLELXYBVtedDa+dIDcA8+UPUJ...,2025-05-18 15:23:53.338592
1,ani,admin,roomTemperatureIQ,None,None,None,True,88,chjd7639383@gmail.com,IMDHToMxUb43ncPXNf4DeWA6meFznZsLDFg9oX+MPomhsc...,2025-05-18 15:23:53.338592
2,testUser1,user,testUser1_cf,testUser1_ac,None,None,True,28,testUser1@gmail.com,EZRAMFJ7IERs8dLVHC1OvpBfDhNWTJFP2uDwme9uZsej3o...,2025-05-18 15:23:53.338592
3,testUser2,user,testUser2_cf,None,None,testUser2_tw,True,69,testUser2@gmail.com,EAA2TudueoURdI+aAgXBGQzG0MVhYs06e0DHBRYNMSLxUI...,2025-05-18 15:23:53.338592
4,testUser3,user,testUser3_cf,None,None,None,True,27,testUser3@gmail.com,ob/xrfD+RWvJZXRGl60eN+rbBowIhW3e07+SK+peLjzInA...,2025-05-18 15:23:53.338592
